In [2]:
# twitter sends its data as a json
import json
# we will convert the json data into a csv
import csv

In [4]:
# numpy's np.nan helps us deal with missing values (and there is a TON of missing values in tweets) 
import numpy as np

In [5]:
# Set the working directory to where the json is located. The output csv will be in this same directory
# Windows users: remember to double-up the backward slashes!
import os
os.chdir("C:\\Users\\PC\\Documents\\GitHub\\Moe_Antar\\Json Parsing Script")

In [6]:
# the fix_encoding function from the FTFY package takes care of weird characters in tweets
import ftfy

In [7]:
import re

In [8]:
# adapted from vincent tatan's youtube guide for NLP
def chwrd(word, text):
    try:
        text = text.lower()
        match = re.search(word, text)
        if match:
            return True
        return False
    except AttributeError:
        return False

# Testing out word_in_text to find a certain word in a text
print(chwrd('hello','Hi there maybe, I would say hello instead'))

True


In [9]:
x = "hello its me"
if chwrd("him",x):
    print("yes")
else:
    print("no")

no


In [10]:
def check_multiple(words,text):
    result = False
    for i in range(len(words)):
        if chwrd(words[i],text) == True:
            result = True
        else:
            pass
    return result

In [12]:
x = "I like Walmart"
y = "I like Bestbuy"
z = "I like ice cream"

check_multiple(["walmart","bestbuy"],x)

True

In [13]:
check_multiple(["walmart","bestbuy"],y)

True

In [14]:
check_multiple(["walmart","bestbuy"],z)

False

In [22]:
def stream_json_file(json_file_name ,competitor_list, supplier_list , bestbuy_list ,output_file_name = "parsed_tweets.csv", stop_at = 1000000000, ):
    # Create an empty csv file that we will append to
    # Create a header row for it
    print("Initalizing Output File:  %s"%output_file_name)
    print("Generating Header Row")
    with open('%s'%output_file_name,'w') as f:
        f.write('id,text,created_at,language,retweet_count,screen_name,country,user_followers_count,time_zone,user_account_location,longitude,lattitude,bestbuy,competitor,supplier,no_category\n') #  Column headers and a trailing new line . MAKE SURE the /n is attached to the last field: eg. text/n
    
    
    tweet_counter = 0
    
    for i in open(json_file_name): 
        
        if tweet_counter > stop_at:
            break
             
        try:
            # Put the data from the current tweet into a list
            # Parse the current tweet
            current_tweet = json.loads(i)
            
            ##################################################################
            
            ## Elements that are 1 level deep ## ##
            
            # Get the id or insert a nan if not found
            if 'id' in current_tweet:
                id = current_tweet['id']
            else:
                id= np.nan
             
            # Get text or insert nan
            
            if 'text' in current_tweet:
                text = current_tweet['text']
                
                # the fix_encoding function from the FTFY package takes care of weird characters
                text = ftfy.fix_encoding(text)
            else:
                text = np.nan
                
                
            # Get created_at or insert nan
            
            if 'created_at' in current_tweet:
                created_at = current_tweet['created_at']
                
            else:
                created_at = np.nan
            
           # Get language or insert nan
           
            if 'lang' in current_tweet:
                language = current_tweet['lang']
     
            else:
                language = np.nan     
            
            
            
            # get retweet count or insert nan
                
            if 'retweet_count' in current_tweet:
                retweet_count = current_tweet['retweet_count']
            else:
                retweet_count = np.nan
                
          ## Elements that are 2 levels deep ### ###
            
            # For elements that are 2 layers deep use != None when searching because javascript uses None as its null operator    
            # get screen name or insert nan
            
            if 'user' in current_tweet and 'screen_name' in current_tweet['user']:
                screen_name = current_tweet['user']['screen_name']
            else:
                screen_name = np.nan
                
            # get country or insert nan

            if current_tweet['place'] != None and current_tweet['place']['country'] != None:
                country = current_tweet['place']['country']
            else:
                country = np.nan
            
            # get the author's follower count or nan

            if current_tweet['user'] != None and current_tweet['user']['followers_count'] != None:
                followers_count = current_tweet['user']['followers_count']
            else:
                followers_count = np.nan
            
            
            # get the timezone or nan
            if current_tweet['user'] != None and current_tweet['user']['time_zone'] != None:
                time_zone = current_tweet['user']['time_zone']
            else:
                time_zone = np.nan
                
            # get the account location or insert nan
            
            if current_tweet['user'] != None and current_tweet['user']['location'] != None:
                account_location = current_tweet['user']['location']
                account_location = ftfy.fix_encoding(account_location)
            else:
                account_location = np.nan
                
            ###### Elements that are 3 levels deep ##################################
            
            if current_tweet['coordinates'] != None and current_tweet['coordinates']['coordinates'] != None and len(current_tweet['coordinates']['coordinates'])==2:
                longitude = current_tweet['coordinates']['coordinates'][0]
            else:
                longitude = np.nan
                
            if current_tweet['coordinates'] != None and current_tweet['coordinates']['coordinates'] != None and len(current_tweet['coordinates']['coordinates'])==2:
                lattitude = current_tweet['coordinates']['coordinates'][1]
            else:
                lattitude = np.nan
                
            ########### Categorization Variables ###############################################
            
            ## True if the word bestbuy is there, False otherwise ###################
            if text != np.nan:
                if check_multiple(bestbuy_list,text):
                    bestbuy = True
                else:
                    bestbuy = False
            else: 
                bestbuy = False
                    
            ### True if the name of a competitor is in there , False otherwise ################

            if text != np.nan:
                if check_multiple(competitor_list,text):
                    competitor = True
                else:
                    competitor = False
            else:
                competitor = False

            ## True if the name of a supplier is in there , False otherwise ################

            if text != np.nan:
                if check_multiple(supplier_list,text):
                    supplier = True
                else:
                    supplier = False
            else:
                supplier = False
                
            ## True if no names of suppliers, competitors, or bestbuy were found
            
            if text != np.nan:
                if bestbuy == False and competitor == False and supplier == False:
                    no_category = True
                else:
                    no_category = False
            
            
            
            
                    
            ######################################################################################################
            # Assemble the row
            cleaned_current_tweet = [id,text,created_at,language, retweet_count, screen_name, country, followers_count,time_zone,account_location,longitude,lattitude,bestbuy,competitor,supplier,no_category]
        
            # Increment the Tweet Counter
            tweet_counter = tweet_counter + 1
            
            # Give the user a progress update
            if tweet_counter % 10000 == 0:
                print(" %d Tweets Parsed so far....." %tweet_counter)
            
            #append the current tweet as a row to the csv
            with open('%s'%output_file_name,'a',newline='') as f:
                writer=csv.writer(f)
                writer.writerow(cleaned_current_tweet)
            
        except:
            pass
    
    print(" ")
    print(" Parsing Complete:    %d Tweets Parsed " %tweet_counter)

In [17]:
my_bestbuy_list= ["best buy","bestbuy","best-buy","geek squad","geeksquad","bbycanada"]
my_competitor_list= ["walmart","wal-mart","wal mart","amazon","prime","visions","ncix","londondrugs",
                                   "london drugs","gamestop","game stop","ebgames","eb games"]
my_supplier_list=["samsung","galaxy","smarthub","apple","mac","iphone","ipad",
                                "tim_cook","appstore","microsoft","xbox","surface",
                                "sony","ps4","playstation","bravia","nintendo","nes","hp",
                                "hewlett","coffeecoaching","lg","lifesgood","g6","canon","dslr","philips","cleanteeth",
                                "google","pixel","android","nexus","chromecast","sharp"]

In [18]:
my_bestbuy_list

['best buy', 'bestbuy', 'best-buy', 'geek squad', 'geeksquad', 'bbycanada']

In [19]:
my_competitor_list

['walmart',
 'wal-mart',
 'wal mart',
 'amazon',
 'prime',
 'visions',
 'ncix',
 'londondrugs',
 'london drugs',
 'gamestop',
 'game stop',
 'ebgames',
 'eb games']

In [20]:
my_supplier_list

['samsung',
 'galaxy',
 'smarthub',
 'apple',
 'mac',
 'iphone',
 'ipad',
 'tim_cook',
 'appstore',
 'microsoft',
 'xbox',
 'surface',
 'sony',
 'ps4',
 'playstation',
 'bravia',
 'nintendo',
 'nes',
 'hp',
 'hewlett',
 'coffeecoaching',
 'lg',
 'lifesgood',
 'g6',
 'canon',
 'dslr',
 'philips',
 'cleanteeth',
 'google',
 'pixel',
 'android',
 'nexus',
 'chromecast',
 'sharp']

In [24]:
# This calls the parsing function. Set the parameters as follows:
## json_file_name = the name of the json file you want to parse eg "industry.json"
## bestbuy_list = a python list containing the bestbuy keywords (e.g ["bestbuy","best-buy"]
## competitor_list = same as above but for competitor keywords
## supplier_list = same as above but for suppliers
## output_file_name = what the csv that is output should be called eg "my_output.csv"
## stop at = how many tweets to parse e.g 1000000 (if you leave it blank it will parse up to a billion tweets)

stream_json_file(json_file_name="industry_wk_2.json",
                 bestbuy_list= my_bestbuy_list,
                 competitor_list= my_competitor_list,
                 supplier_list= my_supplier_list,
                 output_file_name = "clean_wk2_full.csv",
                 stop_at=1000000000)

Initalizing Output File:  clean_wk2_full.csv
Generating Header Row


 10000 Tweets Parsed so far.....


 20000 Tweets Parsed so far.....


 30000 Tweets Parsed so far.....


 40000 Tweets Parsed so far.....


...

 3300000 Tweets Parsed so far.....


 3310000 Tweets Parsed so far.....


 3320000 Tweets Parsed so far.....


 3330000 Tweets Parsed so far.....


 
 Parsing Complete:    3338827 Tweets Parsed 
